In [ ]:
%pip install modin[all]

In [1]:
import dask.dataframe as dd
import gensim
from gensim.models import KeyedVectors
from stellargraph.data import EdgeSplitter
import networkx as nx
from sklearn.linear_model import LogisticRegressionCV

2023-02-28 21:13:37.514221: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-28 21:13:37.663952: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-02-28 21:13:37.668344: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-02-28 21:13:37.668365: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore 

In [2]:
df =  dd.read_csv('../lm-vol/LANL_test.csv', dtype='str', header= None, blocksize="1024MB")

In [ ]:
df =  dd.read_csv('../lm-vol/LANL_anomalies.csv', dtype='str', header= None, blocksize="1024MB")

In [ ]:
df.shape[0].compute()

In [ ]:
graph_structure = (1, 3, 2)
G = nx.Graph()
for index, row in df.iterrows():
    user_node = row[graph_structure[0]]
    ip_node = row[graph_structure[1]]
    service_node = row[graph_structure[2]]
    G.add_nodes_from([ip_node, service_node, user_node])
    G.add_edge(ip_node, user_node)
    G.add_edge(ip_node, service_node)

In [ ]:
nx.write_graphml_lxml(G, "../lm-vol/LANL_test.graphml")

In [ ]:
model = gensim.models.Word2Vec.load('../lm-vol/2_18_500_epochs_word2vec.model')
wv = KeyedVectors.load('../lm-vol/2_18_500_epochs_word2vec.wordvectors', mmap='r')
G = nx.read_graphml('../lm-vol/LANL_train.graphml')
def operator_hadamard(u, v):
    return u * v
node_embeddings = (wv.vectors) 
# Define an edge splitter on the original graph:
edge_splitter_test = EdgeSplitter(G)

# Randomly sample a fraction p=0.1 of all positive links, and same number of negative links, from graph, and obtain the
# reduced graph graph_test with the sampled links removed:
graph_test, edges_train, labels_train = edge_splitter_test.train_test_split(
    p=0.99, method="global"
)
training_edges = [operator_hadamard(wv[edge[0]], wv[edge[1]]) for edge in edges_train]
clf = LogisticRegressionCV(cv=5, max_iter=1000, random_state=0).fit(training_edges, labels_train)
clf.score(training_edges, labels_train)
graph_structure = (1, 3, 2)

In [3]:
df.head(1000)

,0,1,2,3,4,5,6,7,8,9
0,1,ANONYMOUS LOGON@C586,ANONYMOUS LOGON@C586,C1250,C586,NTLM,Network,LogOn,Success,1
1,1,ANONYMOUS LOGON@C586,ANONYMOUS LOGON@C586,C586,C586,?,Network,LogOff,Success,1
2,1,C101$@DOM1,C101$@DOM1,C988,C988,?,Network,LogOff,Success,1
3,1,C1020$@DOM1,SYSTEM@C1020,C1020,C1020,Negotiate,Service,LogOn,Success,1
4,1,C1021$@DOM1,C1021$@DOM1,C1021,C625,Kerberos,Network,LogOn,Success,1
...,...,...,...,...,...,...,...,...,...,...
995,3,C1439$@DOM1,SYSTEM@C1439,C1439,C1439,Negotiate,Service,LogOn,Success,1
996,3,C1441$@DOM1,SYSTEM@C1441,C1441,C1441,Negotiate,Service,LogOn,Success,1
997,3,C1442$@DOM1,SYSTEM@C1442,C1442,C1442,Negotiate,Service,LogOn,Success,1
998,3,C1445$@DOM1,SYSTEM@C1445,C1445,C1445,Negotiate,Service,LogOn,Success,1


In [ ]:
df.npartitions
df = df.persist()

In [ ]:
import multiprocessing as mp
mp.cpu_count()

In [ ]:
# client_to_ip_pred = []
# ip_to_service_pred = []

def func( arg ):
    idx,row = arg
    print(idx)
    try:
        client_embedding = wv[(row[graph_structure[0]])]
        ip_embedding = wv[(row[graph_structure[1]])]
        service_embedding = wv[(row[graph_structure[2]])]
        client_to_ip_embedding = [operator_hadamard(client_embedding, ip_embedding)]
        ip_to_service_embedding = [operator_hadamard(ip_embedding, service_embedding)]
        client_to_ip = clf.predict_proba(client_to_ip_embedding)
        ip_to_service = clf.predict_proba(ip_to_service_embedding)
#         client_to_ip_pred.append(client_to_ip[0][1])
#         ip_to_service_pred.append(ip_to_service[0][1])
        return([client_to_ip[0][1],ip_to_service[0][1]])
    except:
        return([-1,-1])
#         client_to_ip_pred.append(-1)
#         ip_to_service_pred.append(-1)

df['output'] = df.apply(lambda x: func(x), axis=1).compute(scheduler='multiprocessing')

In [ ]:
print(len(test_G.edges))

In [ ]:
test_G = nx.read_graphml('../lm-vol/LANL_test.graphml')
test = []
# max = 100
for index, edge in enumerate(test_G.edges):
#     if index > max:
#         break
    try:
        src = wv[edge[0]]
        dest = wv[edge[1]]
        src_to_dest = [operator_hadamard(src, dest)]
        prob = clf.predict_proba(src_to_dest)
        test.append(prob[0][1])
    except:
        test.append(None)

In [ ]:
print(len(test))

In [ ]:
anomalies = 0
for prob in test:
    if prob is not None and prob <= 0.1:
        anomalies+=1

In [ ]:
print(anomalies)

In [ ]:
18389/515638

In [ ]:
import pickle
import pandas as pd
with open('../lm-vol/2_10_lr.pkl', 'rb') as file:
    clf = pickle.load(file)

In [ ]:
graph_structure = ('source', 'source_computer', 'destination')
inconclusive = 0
n = 0
chunk = pd.read_csv('../lm-vol/redteam_complete.csv')
#     display(chunk)
chunk_invalids = pd.DataFrame()
client_to_ip_pred = []
ip_to_service_pred = []
for index, row in chunk.iterrows():
    try: 
        client_embedding = wv[(row[graph_structure[0]])]
        ip_embedding = wv[(row[graph_structure[1]])]
        service_embedding = wv[(row[graph_structure[2]])]
        client_to_ip_embedding = [operator_hadamard(client_embedding, ip_embedding)]
        ip_to_service_embedding = [operator_hadamard(ip_embedding, service_embedding)]
        client_to_ip = clf.predict_proba(client_to_ip_embedding)
        ip_to_service = clf.predict_proba(ip_to_service_embedding)
        client_to_ip_pred.append(client_to_ip[0][1])
        ip_to_service_pred.append(ip_to_service[0][1])
    except: 
        client_to_ip_pred.append(None)
        ip_to_service_pred.append(None)
        inconclusive += 1
chunk['client_to_ip_pred'] = client_to_ip_pred
chunk['ip_to_service_pred'] = ip_to_service_pred
chunk_anomalies = chunk[(chunk["ip_to_service_pred"].astype(float) <= 0.1) | (chunk["client_to_ip_pred"].astype(float) <= 0.1)]
chunk_invalids = chunk[(chunk["ip_to_service_pred"] == None) & (chunk["client_to_ip_pred"] == None)]
display(chunk_anomalies)

In [ ]:
graph_structure = (1, 3, 2)
client_to_ip_pred = []
ip_to_service_pred = []
inconclusive = 0
for chunk in pd.read_csv('../lm-vol/LANL_test.csv', header=None, chunksize=1000000):
#     display(chunk)
    for index, row in chunk.iterrows():
        try: 
            client_embedding = wv[(row[graph_structure[0]])]
            ip_embedding = wv[(row[graph_structure[1]])]
            service_embedding = wv[(row[graph_structure[2]])]
            client_to_ip_embedding = [operator_hadamard(client_embedding, ip_embedding)]
            ip_to_service_embedding = [operator_hadamard(ip_embedding, service_embedding)]
            client_to_ip = clf.predict_proba(client_to_ip_embedding)
            ip_to_service = clf.predict_proba(ip_to_service_embedding)
            client_to_ip_pred.append(client_to_ip[0][1])
            ip_to_service_pred.append(ip_to_service[0][1])
        except: 
            client_to_ip_pred.append(None)
            ip_to_service_pred.append(None)
            inconclusive += 1
    chunk['client_to_ip_pred'] = client_to_ip_pred
    chunk['ip_to_service_pred'] = ip_to_service_pred
    break

In [ ]:
anomalous_edges = len(chunk[(chunk["ip_to_service_pred"].astype(float) <= 0.1) | (chunk["client_to_ip_pred"].astype(float) <= 0.1)])
normal_edges = len(chunk[(chunk["ip_to_service_pred"].astype(float) > 0.1) & (chunk["client_to_ip_pred"].astype(float) > 0.1)])
print(anomalous_edges/(normal_edges+anomalous_edges))
print(anomalous_edges)
print(normal_edges)

In [ ]:
experiment_names = ['2_18_100_epochs']

def operator_hadamard(u, v):
        return u * v

for name in experiment_names:
    with open(f'../lm-vol/{name}_lr.pkl', 'rb') as file:
        clf = pickle.load(file)
    model = gensim.models.Word2Vec.load(f'../lm-vol/{name}_word2vec.model')
    wv = KeyedVectors.load(f'../lm-vol/{name}_word2vec.wordvectors', mmap='r')
    graph_structure = ('source', 'source_computer', 'destination')
    inconclusive = 0
    n = 0
    red = pd.read_csv('../lm-vol/redteam_complete.csv')
    client_to_ip_pred = []
    ip_to_service_pred = []
    for index, row in red.iterrows():
        try: 
            client_embedding = wv[(row[graph_structure[0]])]
            ip_embedding = wv[(row[graph_structure[1]])]
            service_embedding = wv[(row[graph_structure[2]])]
            client_to_ip_embedding = [operator_hadamard(client_embedding, ip_embedding)]
            ip_to_service_embedding = [operator_hadamard(ip_embedding, service_embedding)]
            client_to_ip = clf.predict_proba(client_to_ip_embedding)
            ip_to_service = clf.predict_proba(ip_to_service_embedding)
            client_to_ip_pred.append(client_to_ip[0][1])
            ip_to_service_pred.append(ip_to_service[0][1])
        except: 
            client_to_ip_pred.append(None)
            ip_to_service_pred.append(None)
            inconclusive += 1
    red['client_to_ip_pred'] = client_to_ip_pred
    red['ip_to_service_pred'] = ip_to_service_pred
    red_anomalies = red[(red["ip_to_service_pred"].astype(float) <= 0.1) | (red["client_to_ip_pred"].astype(float) <= 0.1)]
    red_invalids = red[(red["ip_to_service_pred"] == None) & (red["client_to_ip_pred"] == None)]
    display(red_anomalies)

    test_G = nx.read_graphml('../lm-vol/LANL_test.graphml')
    test = []
    # max = 100
    for index, edge in enumerate(test_G.edges):
    #     if index > max:
    #         break
        try:
            src = wv[edge[0]]
            dest = wv[edge[1]]
            src_to_dest = [operator_hadamard(src, dest)]
            prob = clf.predict_proba(src_to_dest)
            test.append(prob[0][1])
        except:
            test.append(None)
    anomalies = 0
    for prob in test:
        if prob is not None and prob <= 0.1:
            anomalies+=1
    print(anomalies)
    
#     graph_structure = (1, 3, 2)
#     client_to_ip_pred = []
#     ip_to_service_pred = []
#     inconclusive = 0
#     for chunk in pd.read_csv('../lm-vol/LANL_test.csv', header=None, chunksize=1000000):
#     #     display(chunk)
#         for index, row in chunk.iterrows():
#             try: 
#                 client_embedding = wv[(row[graph_structure[0]])]
#                 ip_embedding = wv[(row[graph_structure[1]])]
#                 service_embedding = wv[(row[graph_structure[2]])]
#                 client_to_ip_embedding = [operator_hadamard(client_embedding, ip_embedding)]
#                 ip_to_service_embedding = [operator_hadamard(ip_embedding, service_embedding)]
#                 client_to_ip = clf.predict_proba(client_to_ip_embedding)
#                 ip_to_service = clf.predict_proba(ip_to_service_embedding)
#                 client_to_ip_pred.append(client_to_ip[0][1])
#                 ip_to_service_pred.append(ip_to_service[0][1])
#             except: 
#                 client_to_ip_pred.append(None)
#                 ip_to_service_pred.append(None)
#                 inconclusive += 1
#         chunk['client_to_ip_pred'] = client_to_ip_pred
#         chunk['ip_to_service_pred'] = ip_to_service_pred
#         chunk_anomalies = chunk[(chunk["ip_to_service_pred"].astype(float) <= 0.1) | (chunk["client_to_ip_pred"].astype(float) <= 0.1)]
#         chunk_invalids = chunk[(chunk["ip_to_service_pred"] == None) & (chunk["client_to_ip_pred"] == None)]
#         display(chunk_anomalies)
#         break
    break